In [43]:
import numpy as np
import re
import glob
import itertools
import pickle
import os
import random
import math
import multiprocessing
import matplotlib.pyplot as plt
import imageio
from mpl_toolkits.mplot3d import Axes3D
from pathlib import Path
from tqdm import tqdm_notebook as tqdm
from load_pfm import load_pfm
from utils import get_3d_pos_xy

In [31]:
def bilinear_interp_val(vmap, y, x):
    '''
        bilinear interpolation on a 2D map
    '''
    h, w = vmap.shape
    x1 = int(x)
    x2 = x1 + 1
    x2 = w-1 if x2 > (w-1) else x2
    y1 = int(y)
    y2 = y1 + 1
    y2 = h-1 if y2 > (h-1) else y2
    Q11 = vmap[y1,x1]
    Q21 = vmap[y1,x2]
    Q12 = vmap[y2,x1]
    Q22 = vmap[y2,x2]
    return Q11 * (x2-x) * (y2-y) + Q21 * (x-x1) * (y2-y) + Q12 * (x2-x) * (y-y1) + Q22 * (x-x1) * (y-y1)


def get_3d_pos_xy(y_prime, x_prime, depth, focal_length=1050., w=960, h=540):
    '''
        depth pop up
    '''
    y = (y_prime - h / 2.) * depth / focal_length
    x = (x_prime - w / 2.) * depth / focal_length
    return [x, y, depth]

In [44]:
def gen_datapoint(fname_disparity, fname_disparity_next_frame, fname_disparity_change, fname_optical_flow, \
                  image, image_next_frame, n = 8192, max_cut = 35, focal_length=1050.):

    np.random.seed(601985)

    ##### generate needed data
    disparity_np, _ = load_pfm(fname_disparity)
    disparity_next_frame_np, _ = load_pfm(fname_disparity_next_frame)
    disparity_change_np, _ = load_pfm(fname_disparity_change)
    optical_flow_np, _ = load_pfm(fname_optical_flow)
    rgb_np = imageio.imread(image)[:, :, ::-1] / 255.
    rgb_next_frame_np = imageio.imread(image_next_frame)[:, :, ::-1] / 255.

    depth_np = focal_length / disparity_np
    depth_next_frame_np = focal_length / disparity_next_frame_np
    future_depth_np = focal_length / (disparity_np + disparity_change_np)
    
    h, w = disparity_np.shape

    ##### only take pixels that are near enough
    try:
        depth_requirement = depth_np < max_cut
    except:
        return None

    satisfy_pix1 = np.column_stack(np.where(depth_requirement))
    if satisfy_pix1.shape[0] < n:
        return None
    sample_choice1 = np.random.choice(satisfy_pix1.shape[0], size=n, replace=False)
    sampled_pix1_y = satisfy_pix1[sample_choice1, 0]
    sampled_pix1_x = satisfy_pix1[sample_choice1, 1]

    ##### flow generation
    current_pos1 = np.array([get_3d_pos_xy( sampled_pix1_y[i], sampled_pix1_x[i], \
                                           depth_np[int(sampled_pix1_y[i]), int(sampled_pix1_x[i])] ) for i in range(n)])
    current_rgb1 = np.array([[rgb_np[h-1-int(sampled_pix1_y[i]), int(sampled_pix1_x[i]), 0], \
                              rgb_np[h-1-int(sampled_pix1_y[i]), int(sampled_pix1_x[i]), 1], \
                              rgb_np[h-1-int(sampled_pix1_y[i]), int(sampled_pix1_x[i]), 2]] for i in range(n)])
    sampled_optical_flow_x = np.array([ optical_flow_np[ int( sampled_pix1_y[i] ), int( sampled_pix1_x[i] ) ][0] \
                                       for i in range(n)])
    sampled_optical_flow_y = np.array([ optical_flow_np[ int( sampled_pix1_y[i] ), int( sampled_pix1_x[i] ) ][1] \
                                       for i in range(n)])
    future_pix1_x = sampled_pix1_x + sampled_optical_flow_x
    future_pix1_y = sampled_pix1_y - sampled_optical_flow_y
    future_pos1 = np.array([get_3d_pos_xy( future_pix1_y[i], future_pix1_x[i], \
                                          future_depth_np[int(sampled_pix1_y[i]), int(sampled_pix1_x[i])] ) \
                            for i in range(n)])

    flow = future_pos1 - current_pos1
    
    ##### record original pixels positions
    sampled_pix1_yx = np.column_stack([sampled_pix1_y, sampled_pix1_x])

    ##### only take pixels that are near enough
    try:
        depth_requirement = depth_next_frame_np < max_cut
    except:
        return None

    satisfy_pix2 = np.column_stack(np.where(depth_next_frame_np < max_cut))
    if satisfy_pix2.shape[0] < n:
        return None
    sample_choice2 = np.random.choice(satisfy_pix2.shape[0], size=n, replace=False)
    sampled_pix2_y = satisfy_pix2[sample_choice2, 0]
    sampled_pix2_x = satisfy_pix2[sample_choice2, 1]

    ##### mask, judge whether point move out of fov or occluded by other object after motion
    current_pos2 = np.array([get_3d_pos_xy( sampled_pix2_y[i], sampled_pix2_x[i], \
                                           depth_next_frame_np[int(sampled_pix2_y[i]), int(sampled_pix2_x[i])] ) for i in range(n)])
    current_rgb2 = np.array([[rgb_next_frame_np[h-1-int(sampled_pix2_y[i]), int(sampled_pix2_x[i]), 0], \
                              rgb_next_frame_np[h-1-int(sampled_pix2_y[i]), int(sampled_pix2_x[i]), 1], \
                              rgb_next_frame_np[h-1-int(sampled_pix2_y[i]), int(sampled_pix2_x[i]), 2]] for i in range(n)])
    future_pos1_depth = future_depth_np[sampled_pix1_y, sampled_pix1_x]
    future_pos1_foreground_depth = np.zeros_like(future_pos1_depth)
    valid_mask_fov1 = np.ones_like(future_pos1_depth, dtype=bool)
    for i in range(future_pos1_depth.shape[0]):
        if future_pix1_y[i] > 0 and future_pix1_y[i] < h and future_pix1_x[i] > 0 and future_pix1_x[i] < w:
            future_pos1_foreground_depth[i] = bilinear_interp_val(depth_next_frame_np, future_pix1_y[i], future_pix1_x[i])
        else:
            valid_mask_fov1[i] = False
    valid_mask_occ1 = (future_pos1_foreground_depth - future_pos1_depth) > -5e-1

    mask1 = valid_mask_occ1 & valid_mask_fov1

    return current_pos1, current_pos2, current_rgb1, current_rgb2, flow, mask1, sampled_pix1_yx

In [51]:
def proc_one_scene(s, input_dir_1, input_dir_2, output_dir):
    if s[-1] == '/':
        s = s[:-1]
    dis_split = s.split('/')
    train_or_test = dis_split[-4]
    ABC = dis_split[-3]
    scene_idx = dis_split[-2]
    left_right = dis_split[-1]
    for v in range(6, 15):
        fname = os.path.join(output_dir, train_or_test + '_' + ABC + '_' + scene_idx + '_' + left_right + '_' + str(v).zfill(4) + '-{}'.format(0) + '.npz')

        fname_disparity = os.path.join(input_dir_1, 'disparity', train_or_test, ABC, scene_idx, left_right, str(v).zfill(4) + '.pfm')
        fname_disparity_next_frame = os.path.join(input_dir_1, 'disparity', train_or_test, ABC, scene_idx, left_right, str(v+1).zfill(4) + '.pfm')
        fname_image = os.path.join(input_dir_1, 'frames_finalpass', train_or_test, ABC, scene_idx, left_right, str(v).zfill(4) + '.png')
        fname_image_next_frame = os.path.join(input_dir_1, 'frames_finalpass', train_or_test, ABC, scene_idx, left_right, str(v+1).zfill(4) + '.png')
        fname_disparity_change = os.path.join(input_dir_2, 'disparity_change', train_or_test, ABC, scene_idx, 'into_future', left_right, str(v).zfill(4) + '.pfm')
        L_R = 'L' if left_right == 'left' else 'R'
        fname_optical_flow = os.path.join(input_dir_2, 'optical_flow', train_or_test, ABC, scene_idx, 'into_future', left_right, 'OpticalFlowIntoFuture_' + str(v).zfill(4) + '_' + L_R + '.pfm')

        d = gen_datapoint(fname_disparity, fname_disparity_next_frame, fname_disparity_change, fname_optical_flow, fname_image, fname_image_next_frame, focal_length=1050.)
        if d is not None:
            print(fname)
            np.savez_compressed(fname, points1=d[0], \
                                       points2=d[1], \
                                       color1=d[2], \
                                       color2=d[3], \
                                       flow=d[4], \
                                       valid_mask1=d[5], \
                                       point_yx = d[6])
        else:
            print('N')

In [52]:
INPUT_DIR_1 = '/datasets'
INPUT_DIR_2 = '/data'
OUTPUT_DIR = '/home/tony/datasets/flownet3d_datasets'

if not os.path.exists(OUTPUT_DIR):
    os.system('mkdir -p {}'.format(OUTPUT_DIR))

random.seed('ECCV')

disparity_scenes_train = sorted(list(glob.glob(os.path.join(INPUT_DIR, 'disparity/TRAIN/*/*/*/'))))
disparity_scenes_test = sorted(list(glob.glob(os.path.join(INPUT_DIR, 'disparity/TEST/*/*/*/'))))
disparity_scenes = random.sample(disparity_scenes_train, 2223) + random.sample(disparity_scenes_test, 223)


for s in tqdm(disparity_scenes):
    proc_one_scene(s, INPUT_DIR_1, INPUT_DIR_2, OUTPUT_DIR)


/home/tony/datasets/flownet3d_datasets/TRAIN_C_0634_left_0006-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0634_left_0007-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0634_left_0008-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0634_left_0009-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0634_left_0010-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0634_left_0011-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0634_left_0012-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0634_left_0013-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0634_left_0014-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0681_left_0006-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0681_left_0007-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0681_left_0008-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0681_left_0009-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0681_left_0010-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0

/home/tony/datasets/flownet3d_datasets/TRAIN_C_0006_right_0009-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0006_right_0010-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0006_right_0011-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0006_right_0012-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0006_right_0013-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0006_right_0014-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_B_0456_left_0006-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_B_0456_left_0007-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_B_0456_left_0008-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_B_0456_left_0009-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_B_0456_left_0010-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_B_0456_left_0011-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_B_0456_left_0012-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_B_0456_left_0013-0.npz
/home/tony/datasets/flownet3d_datasets/TRA

/home/tony/datasets/flownet3d_datasets/TRAIN_B_0425_right_0012-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_B_0425_right_0013-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_B_0425_right_0014-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0231_left_0006-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0231_left_0007-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0231_left_0008-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0231_left_0009-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0231_left_0010-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0231_left_0011-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0231_left_0012-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0231_left_0013-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0231_left_0014-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0069_left_0006-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0069_left_0007-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_

/home/tony/datasets/flownet3d_datasets/TRAIN_C_0125_left_0007-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0125_left_0008-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0125_left_0009-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0125_left_0010-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0125_left_0011-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0125_left_0012-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0125_left_0013-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0125_left_0014-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_B_0494_left_0006-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_B_0494_left_0007-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_B_0494_left_0008-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_B_0494_left_0009-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_B_0494_left_0010-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_B_0494_left_0011-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_B_0

/home/tony/datasets/flownet3d_datasets/TRAIN_C_0613_left_0010-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0613_left_0011-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0613_left_0012-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0613_left_0013-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0613_left_0014-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0099_right_0006-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0099_right_0007-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0099_right_0008-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0099_right_0009-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0099_right_0010-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0099_right_0011-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0099_right_0012-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0099_right_0013-0.npz
/home/tony/datasets/flownet3d_datasets/TRAIN_C_0099_right_0014-0.npz
/home/tony/datasets/flownet3d_datasets/

...

/home/tony/datasets/flownet3d_datasets/TEST_B_0111_left_0008-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_B_0111_left_0009-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_B_0111_left_0010-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_B_0111_left_0011-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_B_0111_left_0012-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_B_0111_left_0013-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_B_0111_left_0014-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_C_0126_right_0006-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_C_0126_right_0007-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_C_0126_right_0008-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_C_0126_right_0009-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_C_0126_right_0010-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_C_0126_right_0011-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_C_0126_right_0012-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_C_0126_righ

/home/tony/datasets/flownet3d_datasets/TEST_B_0142_right_0013-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_B_0142_right_0014-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_A_0066_left_0006-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_A_0066_left_0007-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_A_0066_left_0008-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_A_0066_left_0009-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_A_0066_left_0010-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_A_0066_left_0011-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_A_0066_left_0012-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_A_0066_left_0013-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_A_0066_left_0014-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_A_0085_left_0006-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_A_0085_left_0007-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_A_0085_left_0008-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_A_0085_left_0009

/home/tony/datasets/flownet3d_datasets/TEST_C_0029_right_0009-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_C_0029_right_0010-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_C_0029_right_0011-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_C_0029_right_0012-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_C_0029_right_0013-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_C_0029_right_0014-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_B_0070_right_0006-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_B_0070_right_0007-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_B_0070_right_0008-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_B_0070_right_0009-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_B_0070_right_0010-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_B_0070_right_0011-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_B_0070_right_0012-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_B_0070_right_0013-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_B_00

/home/tony/datasets/flownet3d_datasets/TEST_A_0110_right_0014-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_C_0045_left_0006-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_C_0045_left_0007-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_C_0045_left_0008-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_C_0045_left_0009-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_C_0045_left_0010-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_C_0045_left_0011-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_C_0045_left_0012-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_C_0045_left_0013-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_C_0045_left_0014-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_C_0028_left_0006-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_C_0028_left_0007-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_C_0028_left_0008-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_C_0028_left_0009-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_C_0028_left_0010-

/home/tony/datasets/flownet3d_datasets/TEST_A_0077_right_0010-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_A_0077_right_0011-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_A_0077_right_0012-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_A_0077_right_0013-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_A_0077_right_0014-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_B_0133_left_0006-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_B_0133_left_0007-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_B_0133_left_0008-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_B_0133_left_0009-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_B_0133_left_0010-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_B_0133_left_0011-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_B_0133_left_0012-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_B_0133_left_0013-0.npz
/home/tony/datasets/flownet3d_datasets/TEST_B_0133_left_0014-0.npz



In [24]:
INPUT_DIR = '/datasets'
disparity_scenes_train = sorted(list(glob.glob(os.path.join(INPUT_DIR, 'disparity/TRAIN/*/*/*/'))))
disparity_scenes_test = sorted(list(glob.glob(os.path.join(INPUT_DIR, 'disparity/TEST/*/*/*/'))))

In [13]:
disparity_scenes_train

['/datasets/disparity/TRAIN/C/0014/right/',
 '/datasets/disparity/TRAIN/C/0014/left/',
 '/datasets/disparity/TRAIN/C/0027/right/',
 '/datasets/disparity/TRAIN/C/0027/left/',
 '/datasets/disparity/TRAIN/C/0455/right/',
 '/datasets/disparity/TRAIN/C/0455/left/',
 '/datasets/disparity/TRAIN/C/0664/right/',
 '/datasets/disparity/TRAIN/C/0664/left/',
 '/datasets/disparity/TRAIN/C/0484/right/',
 '/datasets/disparity/TRAIN/C/0484/left/',
 '/datasets/disparity/TRAIN/C/0668/right/',
 '/datasets/disparity/TRAIN/C/0668/left/',
 '/datasets/disparity/TRAIN/C/0562/right/',
 '/datasets/disparity/TRAIN/C/0562/left/',
 '/datasets/disparity/TRAIN/C/0274/right/',
 '/datasets/disparity/TRAIN/C/0274/left/',
 '/datasets/disparity/TRAIN/C/0457/right/',
 '/datasets/disparity/TRAIN/C/0457/left/',
 '/datasets/disparity/TRAIN/C/0109/right/',
 '/datasets/disparity/TRAIN/C/0109/left/',
 '/datasets/disparity/TRAIN/C/0441/right/',
 '/datasets/disparity/TRAIN/C/0441/left/',
 '/datasets/disparity/TRAIN/C/0184/right/',

In [8]:
disparity_scenes_test

['/datasets/disparity/TEST/A/0000/left/',
 '/datasets/disparity/TEST/A/0000/right/',
 '/datasets/disparity/TEST/A/0001/left/',
 '/datasets/disparity/TEST/A/0001/right/',
 '/datasets/disparity/TEST/A/0002/left/',
 '/datasets/disparity/TEST/A/0002/right/',
 '/datasets/disparity/TEST/A/0003/left/',
 '/datasets/disparity/TEST/A/0003/right/',
 '/datasets/disparity/TEST/A/0005/left/',
 '/datasets/disparity/TEST/A/0005/right/',
 '/datasets/disparity/TEST/A/0006/left/',
 '/datasets/disparity/TEST/A/0006/right/',
 '/datasets/disparity/TEST/A/0007/left/',
 '/datasets/disparity/TEST/A/0007/right/',
 '/datasets/disparity/TEST/A/0008/left/',
 '/datasets/disparity/TEST/A/0008/right/',
 '/datasets/disparity/TEST/A/0009/left/',
 '/datasets/disparity/TEST/A/0009/right/',
 '/datasets/disparity/TEST/A/0010/left/',
 '/datasets/disparity/TEST/A/0010/right/',
 '/datasets/disparity/TEST/A/0011/left/',
 '/datasets/disparity/TEST/A/0011/right/',
 '/datasets/disparity/TEST/A/0012/left/',
 '/datasets/disparity/T

In [10]:
disparity_scenes_train = sorted(list(Path('/datasets/disparity/TRAIN').glob('*')))
disparity_scenes_train

[PosixPath('/datasets/disparity/TRAIN/A'),
 PosixPath('/datasets/disparity/TRAIN/B'),
 PosixPath('/datasets/disparity/TRAIN/C')]